# Ingest Data Warehouse

### Import


In [1]:
import os
import glob
from typing import Optional
from lib.duckdbcontext import DuckDBContext
import polars as pl
import pyspark
import opendatasets as od
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    explode,
    split,
    udf,
    size,
    regexp_replace,
    when,
    array,
    countDistinct,
    col,
)
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import ast
from pyspark.sql.types import ArrayType, StringType
from tqdm import tqdm
import urllib.request
import duckdb
import json

In [2]:
print(pyspark.__version__)
print(duckdb.__version__)

3.5.1
0.9.2


### Config


In [3]:
duckdb_database = "../orchestration/db/bigdata.duckdb"

#### Setting up Cluster Connection


In [4]:
# Connect to Existing Spark Cluster
# spark = (
#     SparkSession.builder.master("spark://spark:7077")
#     .appName("Spark-ETL")
#     .config("spark.sql.debug.maxToStringFields", 1000)
#     .getOrCreate()
# )

# Connect to local Spark Sessions
spark = (
    SparkSession.builder.master("local")
    .appName("Spark-ETL")
    # .config("spark.sql.debug.maxToStringFields", 1000)
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 16:49:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Add to Data Warehouse

## Initial Data

#### Train


In [5]:
# Get a list of all CSV files that match the pattern
csv_files = glob.glob("../../data/train-*.csv")
print(csv_files)

# Load all CSV files in the data directory into a dataframe
# Specify '\\N' as a null value
# Ignore the header and infer the schema from data
train_spark_df = (
    spark.read.option("header", "true")
    .option("inferSchema", "true")
    .csv("../../data/train-*.csv", nullValue="\\N")
)
# Drop the first column
train_spark_df = train_spark_df.drop("_c0")
train_spark_df.describe().show()
# Print the dataframe
train_spark_df.show(5)

['../../data/train-8.csv', '../../data/train-2.csv', '../../data/train-7.csv', '../../data/train-5.csv', '../../data/train-3.csv', '../../data/train-4.csv', '../../data/train-1.csv', '../../data/train-6.csv']


24/03/17 16:49:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+--------------------+------------------+------------------+------------------+------------------+------------------+
|summary|   tconst|        primaryTitle|     originalTitle|         startYear|           endYear|    runtimeMinutes|          numVotes|
+-------+---------+--------------------+------------------+------------------+------------------+------------------+------------------+
|  count|     7959|                7959|              3971|              7173|               786|              7946|              7169|
|   mean|     NULL|   1231.388888888889|            1128.0|1997.9960964728843|1998.7633587786258|105.68713818273345| 29520.51081043381|
| stddev|     NULL|   954.6947857755001|1038.0438333712118| 21.99534723241901|   21.895931761063| 25.39634772412447|114449.99384975343|
|    min|tt0009369|"Drágớn Báll Z: R...|     'A' gai wak 2|              1918|              1921|                45|            1001.0|
|    max|tt9911196|             Ớútcást|        

#### Directing


In [6]:
# Using Polars to retrieve the directing data
# Load and parse the JSON file
with open("../../data/directing.json") as f:
    data = json.load(f)

movies_polars_df = pl.from_dict(data["movie"]).transpose().rename({"column_0": "movie"})
directors_polars_df = (
    pl.from_dict(data["director"]).transpose().rename({"column_0": "director"})
)
directing_polars_df = pl.concat(
    [
        movies_polars_df,
        directors_polars_df,
    ],
    how="horizontal",
)
directing_polars_df.head(5)

movie,director
str,str
"""tt0003740""","""nm0665163"""
"""tt0008663""","""nm0803705"""
"""tt0009369""","""nm0428059"""
"""tt0009369""","""nm0949648"""
"""tt0010307""","""nm0304098"""


#### Writing


In [7]:
with open("../../data/writing.json") as f:
    data = json.load(f)
writing_json = spark.sparkContext.parallelize(data)
writing_spark_df = spark.read.json(writing_json)
writing_spark_df.show(5)

+---------+---------+
|    movie|   writer|
+---------+---------+
|tt0003740|nm0195339|
|tt0003740|nm0515385|
|tt0003740|nm0665163|
|tt0003740|nm0758215|
|tt0008663|nm0406585|
+---------+---------+
only showing top 5 rows



### Load into DuckDB Database


In [9]:
# DuckDBContext to add pyspark tables to DuckDB
with DuckDBContext(duckdb_database) as ctx:
    ctx.save_to_duckdb(train_spark_df, "imdb_train")
    ctx.show_n("imdb_train", 5)

    ctx.save_to_duckdb(directing_polars_df, "imdb_directors")
    ctx.show_n("imdb_directors", 5)

    ctx.save_to_duckdb(writing_spark_df, "imdb_writing")
    ctx.show_n("imdb_writing", 5)

CREATED TABLE: imdb_train WITH 7959 ROWS!
shape: (5, 8)
┌───────────┬───────────────┬──────────────┬───────────┬─────────┬──────────────┬──────────┬───────┐
│ tconst    ┆ primaryTitle  ┆ originalTitl ┆ startYear ┆ endYear ┆ runtimeMinut ┆ numVotes ┆ label │
│ ---       ┆ ---           ┆ e            ┆ ---       ┆ ---     ┆ es           ┆ ---      ┆ ---   │
│ str       ┆ str           ┆ ---          ┆ i32       ┆ i32     ┆ ---          ┆ f64      ┆ bool  │
│           ┆               ┆ str          ┆           ┆         ┆ i32          ┆          ┆       │
╞═══════════╪═══════════════╪══════════════╪═══════════╪═════════╪══════════════╪══════════╪═══════╡
│ tt0014109 ┆ The Saga of   ┆ null         ┆ 1924      ┆ null    ┆ 183          ┆ 1231.0   ┆ true  │
│           ┆ Gösta Berling ┆              ┆           ┆         ┆              ┆          ┆       │
│ tt0015064 ┆ The Last      ┆ Der letzte   ┆ 1924      ┆ null    ┆ 77           ┆ null     ┆ true  │
│           ┆ Laugh         ┆ Mann 

## Extra Data


### IMDB Datasets


#### Helpers


In [10]:
def get_table_name(file_name: str) -> str:
    return file_name.rsplit(".", 2)[0]


def create_url(endpoint: str) -> str:
    """
    Create Url

    :param str endpoint: download endpoint
    :return str: full url
    """
    return f"https://datasets.imdbws.com/{endpoint}"


def download_file(url, filename):
    print(f"Downloading file: {filename}")

    response = urllib.request.urlopen(url)

    # Get the total file size
    file_size = int(response.headers.get("content-length", 0))

    # Create a tqdm progress bar
    progress = tqdm(total=file_size, unit="iB", unit_scale=True, desc=filename)

    chunk_size = 1024  # you can change this to larger if you want

    with open(filename, "wb") as f:
        while True:
            chunk = response.read(chunk_size)
            if not chunk:
                break
            f.write(chunk)
            progress.update(len(chunk))
    progress.close()

##### Dowloading

Run this cell once, otherwise you'll keep downloading the same files over and over...


In [11]:
extra_imdb = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    # "title.episode.tsv.gz", # we have only movie data
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz",
]

# RUN THIS ONCE!
# Download the files
for ds in extra_imdb:
    # Create an instance of the IMDB class with the desired endpoint
    download_url = create_url(ds)

    filepath = f"../../data/extra/{ds}"  # Local fp
    # Use the function to download the file
    download_file(download_url, filepath)

../../data/extra/name.basics.tsv.gz: 100%|██████████| 263M/263M [00:26<00:00, 9.82MiB/s] 


../../data/extra/title.akas.tsv.gz: 100%|██████████| 330M/330M [00:32<00:00, 10.0MiB/s] 


../../data/extra/title.basics.tsv.gz: 100%|██████████| 186M/186M [00:18<00:00, 9.86MiB/s] 


../../data/extra/title.crew.tsv.gz: 100%|██████████| 70.7M/70.7M [00:07<00:00, 9.68MiB/s]


../../data/extra/title.principals.tsv.gz:   5%|▍         | 23.3M/471M [00:02<00:40, 11.1MiB/s]

## Reading with Spark

RUN THIS ONCE!


In [ ]:
with DuckDBContext(duckdb_database) as ctx:
    train_ids = ctx.conn.execute("SELECT tconst FROM imdb_train").fetchdf()
    train_ids_spark = spark.createDataFrame(train_ids)

    for ds in extra_imdb:
        table_name = f"extra.{get_table_name(ds)}".replace(".", "_")

        # Load a small subset of the data to infer the schema
        subset = spark.read.csv(
            f"../../data/extra/{ds}",
            header=True,
            sep="\t",
            nullValue="\\N",
            inferSchema=True,
        ).limit(1000)

        # Extract the schema from the subset
        schema = subset.schema

        # Load all TSV.GZ files in the data directory into a dataframe with the inferred schema
        spark_df = spark.read.csv(
            f"../../data/extra/{ds}",
            header=True,
            sep="\t",
            nullValue="\\N",
            schema=schema,
        )
        spark_df.show(5)

        spark_df_columns = spark_df.columns

        if "titleId" in spark_df_columns:
            filtered_spark_df = spark_df.join(
                train_ids_spark, train_ids_spark.tconst == spark_df.titleId, "inner"
            )
            filtered_spark_df = filtered_spark_df.drop("titleId")
        elif "knownForTitles" in spark_df_columns:
            # Split the knownForTitles column into multiple rows
            spark_df = spark_df.withColumn(
                "knownForTitles", explode(split(spark_df["knownForTitles"], ","))
            )

            # Select the values that are in both train_ids_spark and spark_df
            filtered_spark_df = spark_df.join(
                train_ids_spark,
                spark_df.knownForTitles == train_ids_spark.tconst,
                "inner",
            )
        elif "tconst" in spark_df_columns:
            filtered_spark_df = spark_df.join(train_ids_spark, "tconst", "inner")

        ctx.save_to_duckdb(filtered_spark_df, table_name)

### Kaggle Data


In [ ]:
# Define a UDF to convert strings to lists
def parse_list(s):
    return s.strip("[]").split(", ")

In [ ]:
# Letterboxd Movie Ratings Data
od.download(
    "https://www.kaggle.com/datasets/samlearner/letterboxd-movie-ratings-data/download?datasetVersionNumber=6",
    data_dir="../../data/extra",
)
# Oscar Award Data
od.download(
    "https://www.kaggle.com/datasets/unanimad/the-oscar-award",
    data_dir="../../data/extra",
)

In [ ]:
# Loop over all CSV files
for file_path in csv_files:
    # Check if 'users_export.csv' or 'ratings_export.csv' is part of the file name
    if "users_export.csv" in file_path or "ratings_export.csv" in file_path:
        # If the file exists, remove it
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"{file_path} removed successfully.")
        else:
            print(f"{file_path} does not exist.")
    else:
        df = (
            spark.read.csv(
                file_path,
                header=True,
                inferSchema=True,
            )
            .limit(5)
            .show(5)
        )

In [ ]:
# Define a UDF to convert strings to lists
def parse_list(s):
    return s.strip("[]").split(", ")


parse_list_udf = udf(parse_list, ArrayType(StringType()))


def process_column(df, column_name):
    # Remove the extra double quotes
    df = df.withColumn(column_name, regexp_replace(df[column_name], '"', ""))
    print(f"Size after removing double quotes: {df.count()} rows")

    # Filter the DataFrame to only include rows where the column is not null
    df = df.filter(col(column_name).isNotNull())
    print(f"Size after filtering nulls: {df.count()} rows")

    # Convert the column to a list
    df = df.withColumn(column_name, parse_list_udf(df[column_name]))
    print(f"Size after converting to list: {df.count()} rows")

    # Check if there are any arrays with multiple values
    multi_value_rows = df.filter(size(df[column_name]) > 1)
    print(
        f"Number of rows with multiple values in {column_name}: {multi_value_rows.count()}"
    )

    # Explode the array into new rows
    df = df.withColumn(column_name[:-1], explode(df[column_name]))
    print(f"Size after exploding list: {df.count()} rows")

    # Drop the original column
    df = df.drop(column_name)
    print(f"Size after dropping original column: {df.count()} rows")

    return df

In [ ]:
# Initialize the DuckDBContext
with DuckDBContext(duckdb_database) as ctx:
    train_ids = ctx.conn.execute("SELECT tconst FROM imdb_train").fetchdf()
    train_ids_spark = spark.createDataFrame(train_ids)

    spark_df = spark.read.csv(
        "../../data/extra/the-oscar-award/the_oscar_award.csv",
        header=True,
        inferSchema=True,
    )
    # Save the DataFrame to DuckDB
    ctx.save_to_duckdb(spark_df, "the_oscar_award")

    # Read Movie Data & drop unnecessary columns
    spark_df = spark.read.csv(
        "../../data/extra/letterboxd-movie-ratings-data/movie_data.csv",
        header=True,
        inferSchema=True,
    ).drop("image_url", "imdb_link", "overview", "tmdb_id", "tmdb_link")

    # Filter on our Train IDs
    filtered_spark_df = spark_df.join(
        train_ids_spark, spark_df.imdb_id == train_ids_spark.tconst, "inner"
    ).drop("imdb_id")
    filtered_spark_df.show(5)

    # Process the genres, production_countries, and spoken_languages columns
    for column in ["genres", "production_countries", "spoken_languages"]:
        filtered_spark_df = process_column(filtered_spark_df, column)

    filtered_spark_df.show(5)

    # Save the DataFrame to DuckDB
    ctx.save_to_duckdb(filtered_spark_df, "letterboxd_movie_ratings_data")